In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
import pickle



In [2]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
movies = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [4]:
movies.info()
credits.info()

# Merge movie with title

In [5]:
merged_movies = movies.merge(credits, on='title')
merged_movies.head(2)

In [6]:
# take only  thoes which are required
merged_movies = merged_movies[['movie_id','title','overview','keywords','genres','cast','crew']]

In [7]:
#current data structure post merge
merged_movies.head(2)

In [8]:
merged_movies.dropna(inplace=True)
merged_movies.isnull().sum()

In [9]:
merged_movies.duplicated().sum()

In [10]:
merged_movies.iloc[0].genres

## Custom formating function

In [11]:
def format(ob):
    l = []
    for i in ast.literal_eval(ob):
        l.append(i['name'])
    return l

In [12]:
def find_director(ob):
    l = []
    for i in ast.literal_eval(ob):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

In [13]:
def format3(ob):
    l = []
    count = 0
    for i in ast.literal_eval(ob):
        if count != 3:
            l.append(i['name'])
            count += 1
        else :
            break
    return l

In [14]:
# assigning to new variable
movies = merged_movies

## Format genres & keywords to save only the names

In [15]:
movies['genres'] = movies['genres'].apply(format)
movies['keywords'] = movies['keywords'].apply(format)

## Format and save 3 cast, save only the names

In [16]:
movies['cast']  = movies['cast'].apply(format3)

## Format and find director

In [17]:
movies['crew'] = movies['crew'].apply(find_director)

## Convert overview to list

In [18]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [19]:
# remove spaces
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])

In [20]:
# create new column tags
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['crew'] + movies['cast'] 

In [21]:
# remove unnecessary columns
movies_df = movies[['movie_id','title','tags']]

In [22]:
# merger all list
movies_df['tags'] = movies_df['tags'].apply( lambda x: " ".join(x) )

In [23]:
# convert to lower case
movies_df['tags'] = movies_df['tags'].apply(lambda x:x.lower()) 

## Converted dataset

In [24]:
movies_df.head()

## Init CountVectorizer

In [25]:
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [26]:
ps = PorterStemmer()

In [27]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)    

## Recommend  function

In [28]:
def recommend(movie):
    movie_index = movies_df[movies_df['title'] ==  movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x: x[1])[1:11]
    
    for i in movie_list:
        print(movies_df.iloc[i[0]].title)

In [29]:
# create tags vectors based on words
vectors = cv.fit_transform(movies_df['tags']).toarray()

In [30]:
# apply stem
movies_df['tags'] =  movies_df['tags'].apply(stem)

In [31]:
# disply feature name & vectors
#cv.get_feature_names()
vectors

In [32]:
# store similarity
similarity = cosine_similarity(vectors)

In [33]:
# sort distance keeping index
sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x: x[1])[1:11]

In [34]:
recommend('Batman')

In [35]:
pickle.dump(movies_df, open('movies.pkl','wb'))